An approach to reformat raw output of AMR-Wind sampling planes to structured data. This code is a bit overkill, as an older version of this file was used to put together TB-sized datasets using `dask`, but we're dealing with much smaller sampling files here.

In [1]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import xarray as xr
from matplotlib.colors import TwoSlopeNorm
from pathlib import Path
import pandas as pd
import h5py

import dask.array as da

Unable to import google auth packages


In [2]:
### Read in data and do some minimal processing
parent_dir = Path('/scratch/orybchuk/wakedynamics/amr-wind-tutorial/03_turbines/post_processing')
outdir = Path(parent_dir, 'reformatted')
outdir.mkdir(exist_ok=True)
t_offset = 14400

fsampling = Path(parent_dir, f'sampling{t_offset}.nc')

# Common parameters

In [3]:
### Set up parameters common to all campaigns
## User inputs
varlist = ['velocityx', 'velocityy', 'velocityz', 'temperature']  # Manually build this
use_dask = False
frequency_output = 1

## Data from simulations
with h5py.File(fsampling) as f_h5py:
    # Deal with time
    time = f_h5py['time'][:]
    time = np.round(time, 4)  # Round to deal with weird float behavior
    timestep = time[1] - time[0]
    
    # List of variables to process
    full_varlist = list(f_h5py['xy-domain'].keys())
for var in varlist:
    assert var in full_varlist, f"Unrecognized variable {var}"
print("varlist:", varlist)

varlist: ['velocityx', 'velocityy', 'velocityz', 'temperature']


# Reformat domain-wide xy data

In [4]:
## Metadata for xy-domain
with h5py.File(fsampling) as f_h5py:
    f_xy = f_h5py['xy-domain']
    ntsteps_xy = f_xy['velocityx'].shape[0]

    da_xy_coords = da.from_array(f_xy['coordinates'], chunks=(-1,-1))
    xy_xcoords = np.sort(np.unique(da_xy_coords[:,0]))
    xy_ycoords = np.sort(np.unique(da_xy_coords[:,1]))
    xy_zcoords = np.sort(np.unique(da_xy_coords[:,2]))
    xy_time = np.round(np.arange(ntsteps_xy)*timestep*frequency_output,2)
xy_coords = {'time':xy_time, 'x': xy_xcoords, 'y': xy_ycoords, 'z': xy_zcoords}

## Reformat into an Xarray Dataset
ds_xy = xr.Dataset(xy_coords)

for var in varlist:
    if use_dask:
        raise NotImplementedError

        # TODO: check the below code
        curr_da = da.from_array(f_xy[var], chunks=chunksize_xy)
        da_reshaped = curr_da.reshape((curr_da.shape[0], len(xy_zcoords), len(xy_ycoords), len(xy_xcoords)))
        ds_xy[var] = (('time', 'z', 'y', 'x'), da_reshaped)
    else:
        with h5py.File(fsampling) as f_h5py:
            curr_arr = f_h5py['xy-domain'][var][:]
        arr_reshaped = curr_arr.reshape((len(time), len(xy_zcoords), len(xy_ycoords), len(xy_xcoords)))
        arr_transposed = np.transpose(arr_reshaped, axes=[0,3,2,1])
        ds_xy[var] = (('time', 'x', 'y', 'z'), arr_transposed)
ds_xy.to_netcdf(Path(outdir,f"xy-domain.nc"))

/home/orybchuk/.conda-envs/daskenv202301/lib/python3.9/site-packages/dask/array/core.py:1701: FutureWarning: The `numpy.sort` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/orybchuk/.conda-envs/daskenv202301/lib/python3.9/site-packages/dask/array/core.py:1701: FutureWarning: The `numpy.sort` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/orybchuk/.conda-envs/daskenv202301/lib/python3.9/site-packages/dask/array/core.py:1701: FutureWarning: The `numpy.sort` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(


# Reformat domain-wide xz data

In [5]:
## Metadata for met masts
with h5py.File(fsampling) as f_h5py:
    f_xz = f_h5py['xz-domain']
    ntsteps_xz = f_xz['velocityx'].shape[0]

    da_xz_coords = da.from_array(f_xz['coordinates'], chunks=(-1,-1))
    xz_xcoords = np.sort(np.unique(da_xz_coords[:,0]))
    xz_ycoords = np.sort(np.unique(da_xz_coords[:,1]))
    xz_zcoords = np.sort(np.unique(da_xz_coords[:,2]))
    xz_time = np.round(np.arange(ntsteps_xz)*timestep*frequency_output,2)
xz_coords = {'time':xz_time, 'x': xz_xcoords, 'y': xz_ycoords, 'z': xz_zcoords}

## Reformat into an Xarray Dataset
ds_xz = xr.Dataset(xz_coords)

for var in varlist:
    if use_dask:
        raise NotImplementedError

        # TODO: check the below code
        curr_da = da.from_array(f_xz[var], chunks=chunksize_xz)
        da_reshaped = curr_da.reshape((curr_da.shape[0], len(xz_zcoords), len(xz_ycoords), len(xz_xcoords)))
        ds_xz[var] = (('time', 'z', 'y', 'x'), da_reshaped)
    else:
        with h5py.File(fsampling) as f_h5py:
            curr_arr = f_h5py['xz-domain'][var][:]
        arr_reshaped = curr_arr.reshape((len(time), len(xz_zcoords), len(xz_ycoords), len(xz_xcoords)))
        arr_transposed = np.transpose(arr_reshaped, axes=[0,3,2,1])
        ds_xz[var] = (('time', 'x', 'y', 'z'), arr_transposed)
ds_xz.to_netcdf(Path(outdir,f"xz-domain.nc"))

/home/orybchuk/.conda-envs/daskenv202301/lib/python3.9/site-packages/dask/array/core.py:1701: FutureWarning: The `numpy.sort` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/orybchuk/.conda-envs/daskenv202301/lib/python3.9/site-packages/dask/array/core.py:1701: FutureWarning: The `numpy.sort` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
/home/orybchuk/.conda-envs/daskenv202301/lib/python3.9/site-packages/dask/array/core.py:1701: FutureWarning: The `numpy.sort` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  warnings.warn(
